### Imports

In [ ]:
import pandas as pd
import os
from scipy import stats

from delong_auc import *

import gensim
from gensim.models import Doc2Vec

import joblib
from sklearn.metrics import roc_auc_score, roc_curve

from process_text import text_to_vectors

import matplotlib.pyplot as plt
%matplotlib inline

### 1. Read files

In [ ]:
predictions = pd.read_csv("logs/predictions.csv", sep=";")

### 2. Evaluate cross validation prediction (internal)

Compute area under curve and standard deviation using DeLong method

In [ ]:
# Create empty figure
plt.figure(figsize=(7, 5), dpi=450)

# Track values over multiple folds
aucs = []
auc_vars = []

# Determine number of folds
no_folds = predictions['fold_number'].nunique()

# For each fold
for i in np.arange(no_folds)+1:
    
    # Select subset of dataframe corresponding to fold
    predictions_fold = predictions[predictions['fold_number'] == i]
    
    # Compute area under curve and variance based on DeLong method
    auc, var = delong_roc_variance(predictions_fold['true_label'], 
                                   predictions_fold['probability'])

    # Track total
    aucs.append(auc)
    auc_vars.append(var)
    
    # Compute FPR and TPR rates for plotting
    fpr, tpr, thresholds = roc_curve(predictions_fold['true_label'], 
                                     predictions_fold['probability'])        
    
    # Add to plot
    plt.plot(fpr, tpr, label="Fold {} (AUC={:.3f})".format(i, auc))
    
# Sampling distribution of the mean 
auc_mean = np.mean(aucs)
auc_var = np.mean(auc_vars)
auc_ste = np.sqrt(auc_var) / np.sqrt(no_folds)
    
plt.plot([0, 1], [0, 1], '--')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.title('AUROC Site x = {:.3f}'.format(auc_mean))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")    
    
# Compute area under curve and variance based on DeLong method
auc_delong, var_delong = delong_roc_variance(predictions['true_label'], 
                               predictions['probability'])
    
print("Auc = {:.5f}".format(auc_mean))
print("Var = {:.5f}".format(auc_var))    
print("Ste = {:.5f}".format(auc_ste))    
print("95% CI = {}".format(stats.norm.ppf([0.025, 0.975],loc=auc_mean, scale=auc_ste)))

Compute binary classification statistics (True Positives, True Negatives, False Positives, False Negatives, etc). 

In [ ]:
# Determine a binary cutoff for a group of predictions
def binary_cutoff(group):
    
    # Ratio of admissions with a positive outcome in this group
    ratio = 1 - (sum(group['true_label']) / len(group['true_label']))
    
    # Determine threshold based on this ratio
    threshold = sorted(group['probability'])[int(ratio * len(group))]
    group['binary_prediction'] = (group['probability'] > threshold)
    
    # Return
    return(group)

# A binary cutoff is determined for each fold
predictions = predictions.groupby("fold_number").apply(binary_cutoff)

# Show 2x2 contingency table
pd.crosstab(predictions['true_label'], predictions['binary_prediction'])

### Evaluate external models on internal dataset

In [ ]:
have_external_model = False

In [ ]:
if have_external_model:

    # Read external paragraph2vec and svm models
    p2v_model_external = Doc2Vec.load("models_external/paragraph2vec_model")
    svm_model_external = joblib.load("models_external/svm_model")

    # Read processed notes
    notes = pd.read_csv("data/processed/notes.csv", sep=";")

In [ ]:
if have_external_model:
    # Obtain vectors of notes using external paragraph2vec model
    note_vectors_external = text_to_vectors(notes, 'words_stemmed', p2v_model_external, no_reps=10)

    # Predict probabilities using external classification model
    probability_external = svm_model_external.predict_proba(note_vectors_external)[:, 1]

    # Create dataframe with predictions
    predictions_external = pd.DataFrame({'probability' : probability_external, 
                                     'true_label'  : notes['outcome'].map({0 : False, 1 : True})})

    # All predictions in same 'fold'
    predictions_external['fold_number'] = 1

    # Determine binary cutoff
    predictions_external = predictions_external.groupby("fold_number").apply(binary_cutoff)

    # Compute area under curve and covariance based on DeLong method
    auc_external, auc_var_external = delong_roc_variance(predictions_external['true_label'], 
                                                 predictions_external['probability'])

    print("External auc = {:.3f}".format(auc_external))
    print("External ste = {:.3f}".format(np.sqrt(auc_var_external)))
    print("External 95% CI = {}".format(stats.norm.ppf([0.025, 0.975],loc=auc_external, scale=np.sqrt(auc_var_external))))

    predictions_external.to_csv("logs/predictions_external.csv", sep=";", index=False)

# Metric analysis

In [ ]:
import sklearn

In [ ]:
PRED_THR = 0.06
y_pred = [el > PRED_THR for el in predictions.probability.to_list()]
y_true = predictions.true_label.to_list()
recall = sklearn.metrics.recall_score(y_true, y_pred)
print(recall)

Now according to the curve provided above, the false positive rate can be estimated visually to be 40%

In [ ]:
VISUAL_FPR = 0.4

In [ ]:
fp = 0
tn = 0
for i in range(len(y_true)):
    if y_true[i]:
        continue
    tn += 1
    if y_pred[i]:
        fp += 1
fpr = float(fp) / tn

In [ ]:
if abs(fpr - VISUAL_FPR) > VISUAL_FPR / 8:
    print("ERROR: big discrepancy in false-positive rate")
else:
    print("SUCCESS: false-positive rate is as we expected")

This means that if we choose to catch a lot of violence incidents, we will also predict violence incidents incorrectly for a relatively large number of patients. Now remember we have very imbalanced datasets:

In [ ]:
original_incidents = len([el for el in predictions.true_label.to_list() if el == True])
new_incidents = (1 - recall) * original_incidents
original_innocent = len([el for el in predictions.true_label.to_list() if el == False])
false_incidents = fpr * original_innocent
avoided_incidents = recall * original_incidents
print(str(int(avoided_incidents)) + " incidents avoided")
print(str(int(new_incidents)) + " incidents remain")
print(str(int(false_incidents))  + " cases incorrectly labeled as aggressive")
print("Precision = " + str(sklearn.metrics.precision_score(y_true, y_pred)))
print("F1 = " + str(sklearn.metrics.f1_score(y_true, y_pred)))

If the precision is really low, incorrectly classify cases as violent too often

With the parameters chosen by Vincent:

In [ ]:
official_recall = sklearn.metrics.recall_score(predictions.true_label.to_list(), predictions.binary_prediction.to_list())
official_fp = len(predictions[(predictions['true_label'] == False) & (predictions['binary_prediction'] == True)])
print(str(int(official_recall * original_incidents)) + " incidents avoided")
print(str(int((1 - official_recall) * original_incidents)) + " incidents remain")
print(str(int(official_fp))  + " cases incorrectly labeled as aggressive")
print('Precision', sklearn.metrics.precision_score(predictions.true_label, predictions.binary_prediction))
print('F1', sklearn.metrics.f1_score(predictions.true_label, predictions.binary_prediction))

This would mean less incidents are avoided but also fewer innocents are labelled as aggressive. The threshold should be decided one you know what each entails. For example, if aggression is extremely bad (because people get hurt), then we would go to a model with more false positives. If, on the other hand, aggression only means that more personnel have to be made available to deal with the violent patients, but on the other hand false positives are a problem because we have to keep track of them, then we'd want to keep the false positive rate also somewhat low. So, indeed, we need a metric that keeps both at reasonable levels. That's why maybe the area under the curve is a good metric. What's the maximum F1 score? And what is the threshold?

The expert (Floor Scheepers) says it's better to have a high recall (avoid many violence incidents), as I had suspected above

In [ ]:
def get_labels(threshold):
    y_pred = predictions['probability'].apply(lambda x : x > threshold)
    return y_pred

In [ ]:
# Discretise thresholds
thresholds = np.arange(0, 1, 0.001)
y_true = predictions.true_label.to_list()
f1s = []
for thr in thresholds:
    y_pred = get_labels(thr)
    f1s.append(sklearn.metrics.f1_score(y_true, y_pred))
    
max_f1 = 0
max_f1_index = -1
for i in range(len(f1s)):
    if f1s[i] > max_f1:
        max_f1_index = i
        max_f1 = f1s[i]
print('threshold', thresholds[max_f1_index])
print('maximum F1',max_f1)
print('precision', sklearn.metrics.precision_score(y_true, get_labels(thresholds[max_f1_index])))
print('recall', sklearn.metrics.recall_score(y_true, get_labels(thresholds[max_f1_index])))

So the best F1 is close to the official value. The baseline precision and recall and accuracy are

p = f

r = f

acc = 2f^2-2f+1

where f is the fraction of positives (incidence)

In [ ]:
p_baseline = float(original_incidents) / (original_incidents + original_innocent)
r_baseline = p_baseline
acc_baseline = 2 * p_baseline ** 2 - 2 * p_baseline + 1

In [ ]:
print('baseline precision and recall', p_baseline)

One nice conclusion of this is that, even in the high-recall-low-precision scenario that I wrote above, this model performs better than the baseline